In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, label_binarize, LabelEncoder
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, auc as sklearn_auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings

import statsmodels.api as sm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, mean_absolute_error, mean_squared_error, r2_score

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)


random_state = 42


files_bfi2 = [
    "Human_GPT3.5_persona_BFI2.xlsx",
    "Human_GPT3.5_shape_BFI2.xlsx",
    "Human_GPT4_persona_BFI2.xlsx",
    "Human_GPT4_shape_BFI2.xlsx",
    'Human_LLaMA3_persona_BFI2.xlsx',
    "Human_LLaMA3_shape_BFI2.xlsx",
]

files_hexao= [
    "Human_GPT3.5_persona_HEXAO.xlsx",
    "Human_GPT3.5_shape_HEXAO.xlsx",
    "Human_GPT4_persona_HEXAO.xlsx",
    "Human_GPT4_shape_HEXAO.xlsx",
    "Human_LLaMA3_persona_HEXAO.xlsx",
    "Human_LLaMA3_shape_HEXAO.xlsx"
]

# Seperate

In [13]:
def clean(df, item_counts):
    columns_selected = [f"item{i}" for i in range(1, item_counts+1)]  # Ensure your DataFrame has these columns

    # clean errored data
    X = df[columns_selected]  # Features (34 columns)
    mask = X.apply(lambda x: x.between(1, 5)).all(axis=1)
    df = df[mask]

    # Calculate mean and standard deviation
    df['mean'] = df[columns_selected].mean(axis=1)
    df['std'] = df[columns_selected].std(axis=1)
    # print(df["label"].value_counts())
    return df

In [3]:
def random_regression_with_kfold(df, k=5, drawing=False):
    X = df[['mean', 'std']]
    y = df["label"]

    # print(X)
    # print("------------")
    # print(y)
    # print("------------")
    
    # Encode the target variable
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    # print(y_encoded) 
    
    # Initialize the logistic regression model (multinomial for multiple categories)
    model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
    
    # K-Fold Cross Validation
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    auc_scores = []
    confusion_matrices = []
    mean_p_values = []
    std_p_values = []
    mae_scores = []
    mse_scores = []
    r2_scores = []
    rmse_scores = []
    mean_coeffs = []
    std_coeffs = []

    for train_index, test_index in kf.split(X, y_encoded):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred, average='macro'))
        recall_scores.append(recall_score(y_test, y_pred, average='macro'))
        f1_scores.append(f1_score(y_test, y_pred, average='macro'))
        mae_scores.append(mean_absolute_error(y_test, y_pred))
        mse_scores.append(mean_squared_error(y_test, y_pred))
        r2_scores.append(r2_score(y_test, y_pred))
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
        
        try:
            auc_scores.append(roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr', average='macro'))
        except ValueError:
            auc_scores.append(None)
        
        confusion_matrices.append(confusion_matrix(y_test, y_pred))
        
        # Significance levels of each feature within each fold
        X_with_const = sm.add_constant(X_train)
        logit_model = sm.MNLogit(y_train, X_with_const)
        result = logit_model.fit()
        p_values = result.pvalues

        mean_coeffs.append(result.params.loc['mean', 0])
        std_coeffs.append(result.params.loc['std', 0])
        mean_p_values.append(p_values.loc['mean', 0])
        std_p_values.append(p_values.loc['std', 0])

    # print("------------------------------")
    # print("accuracy_scores, ", accuracy_scores)
    # print("------------------------------")
    # print("recall_scores, ", recall_scores)
    
    # Calculate mean and std scores across all folds
    mean_accuracy = np.mean(accuracy_scores)
    std_accuracy = np.std(accuracy_scores)
    mean_precision = np.mean(precision_scores)
    std_precision = np.std(precision_scores)
    mean_recall = np.mean(recall_scores)
    std_recall = np.std(recall_scores)
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    mean_auc = np.mean([score for score in auc_scores if score is not None])
    std_auc = np.std([score for score in auc_scores if score is not None])
    mean_p_value_mean = np.mean(mean_p_values)
    std_p_value_mean = np.std(mean_p_values)
    mean_p_value_std = np.mean(std_p_values)
    std_p_value_std = np.std(std_p_values)
    mean_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)
    mean_mse = np.mean(mse_scores)
    std_mse = np.std(mse_scores)
    mean_r2 = np.mean(r2_scores)
    std_r2 = np.std(r2_scores)
    mean_rmse = np.mean(rmse_scores)
    std_rmse = np.std(rmse_scores)
    mean_coeff_mean = np.mean(mean_coeffs)
    std_coeff_mean = np.std(mean_coeffs)
    mean_coeff_std = np.mean(std_coeffs)
    std_coeff_std = np.std(std_coeffs)
    
    if drawing:
        plt.figure(figsize=(8, 6))
        sns.heatmap(np.mean(confusion_matrices, axis=0), annot=True, fmt='.2f', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
        plt.show()
    
    return {
        "Mean Accuracy": mean_accuracy,
        "Std Accuracy": std_accuracy,
        "Mean Precision": mean_precision,
        "Std Precision": std_precision,
        "Mean Recall": mean_recall,
        "Std Recall": std_recall,
        "Mean F1 Score": mean_f1,
        "Std F1 Score": std_f1,
        "Mean AUC": mean_auc,
        "Std AUC": std_auc,
        "Confusion Matrices": confusion_matrices,
        "Mean P-Value (mean)": mean_p_value_mean,
        "Std P-Value (mean)": std_p_value_mean,
        "Mean P-Value (std)": mean_p_value_std,
        "Std P-Value (std)": std_p_value_std,
        "Mean Absolute Error (MAE)": mean_mae,
        "Std Absolute Error (MAE)": std_mae,
        "Mean Squared Error (MSE)": mean_mse,
        "Std Squared Error (MSE)": std_mse,
        "R-squared (R²) Score": mean_r2,
        "Std R-squared (R²) Score": std_r2,
        "Root Mean Squared Error (RMSE)": mean_rmse,
        "Std Root Mean Squared Error (RMSE)": std_rmse,
        "Mean Coefficient (mean)": mean_coeff_mean,
        "Std Coefficient (mean)": std_coeff_mean,
        "Mean Coefficient (std)": mean_coeff_std,
        "Std Coefficient (std)": std_coeff_std,
    }



def get_n_rounds(files, items_count, n=1):
    all_results = []

    for _ in range(n):
        round_results = []
        for file in files:
            df = pd.read_excel(f'data/{file}')
            df = clean(df, items_count)

            df_lm = df[df["label"] == "LM"].reset_index(drop=True)
            print(len(df_lm))
            df_human = df[df["label"] == "Human"].reset_index(drop=True)
            df_human = df_human.sample(300).reset_index(drop=True)  # we only have 300 for LM
            
            df = pd.concat([df_lm, df_human], axis=0)
            
            res = random_regression_with_kfold(df, k=5, drawing=False)
            res["source"] = file
            round_results.append(res)
        all_results.append(pd.DataFrame(round_results))


    
    # Calculate average of each metric over all rounds
     # Concatenate all results and group by 'source' to calculate mean and std of numeric columns
    all_results_df = pd.concat(all_results)
    # print(all_results_df)
    numeric_cols = all_results_df.select_dtypes(include=np.number).columns
    averaged_results = all_results_df.groupby('source')[numeric_cols].mean().reset_index()

    return averaged_results, all_results_df



## bfi2

In [4]:
df_1, original_df = get_n_rounds(files_bfi2, 60, 50)

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


299
Optimization terminated successfully.
         Current function value: 0.258930
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.237279
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.259733
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.264146
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.248571
         Iterations 8
300
Optimization terminated successfully.
         Current function value: 0.509022
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.543078
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.547195
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.529364
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.5

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


299
Optimization terminated successfully.
         Current function value: 0.225679
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.239298
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.238921
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.252206
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.248777
         Iterations 8
300
Optimization terminated successfully.
         Current function value: 0.490717
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.515499
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.526913
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.500682
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.5

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


...

299
Optimization terminated successfully.
         Current function value: 0.270681
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.254049
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.287474
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.292829
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.297218
         Iterations 8
300
Optimization terminated successfully.
         Current function value: 0.525084
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.536325
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.549027
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.538961
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.5

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


299
Optimization terminated successfully.
         Current function value: 0.256814
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.234808
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.246359
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.277233
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.263120
         Iterations 8
300
Optimization terminated successfully.
         Current function value: 0.519274
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.546804
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.559173
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.533351
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.5

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


299
Optimization terminated successfully.
         Current function value: 0.224666
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.233382
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.229349
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.267278
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.241647
         Iterations 8
300
Optimization terminated successfully.
         Current function value: 0.489931
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.522015
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.516480
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.505750
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.5

##  why persona gpt 4 is better for bfi2

In [5]:
df_1[["source", 'Mean Accuracy', 'Std Accuracy', 'Mean Precision', 'Std Precision',
       'Mean Recall', 'Std Recall', 'Mean F1 Score', 'Std F1 Score', 
      "Mean Coefficient (mean)", "Std Coefficient (mean)", "Mean Coefficient (std)", "Std Coefficient (std)"]]

,source,Mean Accuracy,Std Accuracy,Mean Precision,Std Precision,Mean Recall,Std Recall,Mean F1 Score,Std F1 Score,Mean Coefficient (mean),Std Coefficient (mean),Mean Coefficient (std),Std Coefficient (std)
0,Human_GPT3.5_persona_BFI2.xlsx,0.902096,0.022765,0.904203,0.022362,0.902125,0.022733,0.901958,0.022828,-0.714102,0.413490,-12.376621,0.735320
1,Human_GPT3.5_shape_BFI2.xlsx,0.742967,0.039621,0.745415,0.038418,0.742967,0.039621,0.742189,0.040271,-2.399038,0.135917,-3.777379,0.206613
2,Human_GPT4_persona_BFI2.xlsx,0.971133,0.012216,0.971619,0.012011,0.971133,0.012216,0.971123,0.012224,0.639120,0.373054,-15.313307,0.877887
3,Human_GPT4_shape_BFI2.xlsx,0.738867,0.034782,0.742752,0.033681,0.738867,0.034782,0.737699,0.035644,-1.432213,0.112242,-3.390011,0.244467
4,Human_LLaMA3_persona_BFI2.xlsx,0.649167,0.028736,0.650130,0.028574,0.649167,0.028736,0.648534,0.029063,-0.410136,0.146727,-2.966418,0.221542
5,Human_LLaMA3_shape_BFI2.xlsx,0.645894,0.038234,0.651712,0.038883,0.645380,0.038464,0.641650,0.040608,-1.811143,0.105345,-0.152922,0.090749


In [7]:
original_df[["source", 'Mean Accuracy', 'Std Accuracy', 'Mean Precision', 'Std Precision',
       'Mean Recall', 'Std Recall', 'Mean F1 Score', 'Std F1 Score', 
      "Mean Coefficient (mean)", "Std Coefficient (mean)", "Mean Coefficient (std)", "Std Coefficient (std)"]]

,source,Mean Accuracy,Std Accuracy,Mean Precision,Std Precision,Mean Recall,Std Recall,Mean F1 Score,Std F1 Score,Mean Coefficient (mean),Std Coefficient (mean),Mean Coefficient (std),Std Coefficient (std)
0,Human_GPT3.5_persona_BFI2.xlsx,0.904832,0.017212,0.905714,0.017703,0.904859,0.017203,0.904789,0.017189,-1.089949,0.452433,-12.116572,0.628860
1,Human_GPT3.5_shape_BFI2.xlsx,0.725000,0.052175,0.727131,0.051807,0.725000,0.052175,0.724207,0.052733,-2.392568,0.183776,-3.765113,0.236472
2,Human_GPT4_persona_BFI2.xlsx,0.983333,0.005270,0.983602,0.005277,0.983333,0.005270,0.983331,0.005271,-1.134488,0.417033,-17.806330,1.162258
3,Human_GPT4_shape_BFI2.xlsx,0.736667,0.032318,0.740263,0.029874,0.736667,0.032318,0.735453,0.033541,-1.390354,0.081800,-3.247481,0.265219
4,Human_LLaMA3_persona_BFI2.xlsx,0.646667,0.023921,0.647620,0.023585,0.646667,0.023921,0.646024,0.024295,-0.445329,0.127024,-2.757742,0.220462
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Human_GPT3.5_shape_BFI2.xlsx,0.745000,0.036742,0.746273,0.036176,0.745000,0.036742,0.744598,0.037093,-2.492128,0.170357,-4.100739,0.228202
2,Human_GPT4_persona_BFI2.xlsx,0.968333,0.016997,0.968863,0.016503,0.968333,0.016997,0.968319,0.017012,0.528335,0.547442,-13.938339,0.745801
3,Human_GPT4_shape_BFI2.xlsx,0.745000,0.030092,0.748739,0.027528,0.745000,0.030092,0.743839,0.031218,-1.417583,0.074625,-3.183032,0.286762
4,Human_LLaMA3_persona_BFI2.xlsx,0.640000,0.037417,0.641071,0.036393,0.640000,0.037417,0.638940,0.038658,-0.266078,0.183429,-2.682308,0.277993


In [14]:
file = files_bfi2[2]
items_count = 60

file

'Human_GPT4_persona_BFI2.xlsx'

In [20]:
df = pd.read_excel(f'data/{file}')
df = clean(df, items_count)

df_lm = df[df["label"] == "LM"].reset_index(drop=True)
print(len(df_lm))
df_human = df[df["label"] == "Human"].reset_index(drop=True)
df_human = df_human.sample(300).reset_index(drop=True)  # we only have 300 for LM

df = pd.concat([df_lm, df_human], axis=0)
df.head()

300


,item1,item2,item3,item4,item5,item6,item7,item8,item9,item10,...,item54,item55,item56,item57,item58,item59,item60,label,mean,std
0,3,3,3,3,5,3,5,3,3,5,...,3,3,4,3,3,3,3,LM,3.400000,0.717812
1,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,LM,3.033333,0.181020
2,3,3,3,3,4,3,3,3,3,4,...,3,3,3,3,3,3,3,LM,3.200000,0.546173
3,3,3,3,2,4,3,3,3,5,3,...,2,3,4,4,3,3,4,LM,3.050000,0.699273
4,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,LM,3.000000,0.000000


In [25]:
X = df[['mean', 'std']]
y = df["label"]


# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# print(y_encoded) 

# Initialize the logistic regression model (multinomial for multiple categories)
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model.fit(X, y)

LogisticRegression(max_iter=200, multi_class='multinomial')

In [27]:
"""
所以mean是不太相关的
std比较相关

"""
model.coef_

array([[-0.05779309, -4.45186844]])

In [28]:
model.intercept_

array([4.20869213])

In [52]:
file = files_bfi2[0]
items_count = 60
df = pd.read_excel(f'data/{file}')
df = clean(df, items_count)

df_lm = df[df["label"] == "LM"].reset_index(drop=True)
print(len(df_lm))
df_human = df[df["label"] == "Human"].reset_index(drop=True)
df_human = df_human.sample(300).reset_index(drop=True)  # we only have 300 for LM

df = pd.concat([df_lm, df_human], axis=0)


item, mean_cor, std_cor = [], [], []
for i in range(1, 61):
    col = f"item{i}"
    item.append(col)

    mean_cor.append(df[col].corr(df["mean"]))
    std_cor.append(df[col].corr(df["std"]))

corrs = pd.DataFrame({"feature":item, "mean_cor_0":mean_cor, "std_cor_0":std_cor})
corrs


for j in range(1, len(files_bfi2)):
    file = files_bfi2[j]
    df = pd.read_excel(f'data/{file}')
    df = clean(df, items_count)
    
    df_lm = df[df["label"] == "LM"].reset_index(drop=True)
    df_human = df[df["label"] == "Human"].reset_index(drop=True)
    df_human = df_human.sample(300).reset_index(drop=True)  # we only have 300 for LM
    
    df = pd.concat([df_lm, df_human], axis=0)
    
    
    item, mean_cor, std_cor = [], [], []
    for i in range(1, 61):
        col = f"item{i}"
        item.append(col)
    
        mean_cor.append(df[col].corr(df["mean"]))
        std_cor.append(df[col].corr(df["std"]))
    
    corrs_i = pd.DataFrame({"feature":item, f"mean_cor_{j}":mean_cor, f"std_cor_{j}":std_cor})
    corrs_i

    corrs = pd.concat([corrs, corrs_i], axis=1)
corrs

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


299


,feature,mean_cor_0,std_cor_0,feature,mean_cor_1,std_cor_1,feature,mean_cor_2,std_cor_2,feature,mean_cor_3,std_cor_3,feature,mean_cor_4,std_cor_4,feature,mean_cor_5,std_cor_5
0,item1,0.420127,-0.002311,item1,0.511529,-0.043173,item1,0.385103,0.005115,item1,0.525621,-0.053894,item1,0.442645,0.085188,item1,0.564430,-0.029455
1,item2,0.422935,0.146981,item2,0.570122,0.167164,item2,0.616402,0.625842,item2,0.727919,0.196623,item2,0.489120,0.094305,item2,0.671411,0.036005
2,item3,0.372631,0.112728,item3,0.496966,0.116391,item3,0.441356,0.240883,item3,0.480852,0.224400,item3,0.413903,0.128685,item3,0.552341,0.181806
3,item4,-0.056454,-0.122477,item4,-0.122298,-0.096436,item4,-0.142050,-0.295265,item4,-0.340241,-0.075787,item4,-0.191917,-0.213511,item4,-0.384435,-0.034265
4,item5,0.425170,0.292513,item5,0.436146,0.046116,item5,0.423498,0.178780,item5,0.478572,0.172717,item5,0.428959,-0.036508,item5,0.685287,0.018713
5,item6,0.260625,-0.144867,item6,0.088136,-0.038351,item6,0.217289,-0.035426,item6,0.351878,-0.002612,item6,0.331191,-0.040661,item6,0.304427,0.075503
6,item7,0.221084,-0.134368,item7,0.492098,0.105315,item7,0.626024,0.773179,item7,0.684828,0.241586,item7,0.520295,0.043737,item7,0.723124,0.111723
7,item8,0.428671,0.003103,item8,0.567695,0.101452,item8,0.493151,0.206267,item8,0.654139,0.221210,item8,0.488122,0.075285,item8,0.621294,0.180354
8,item9,-0.085707,-0.040836,item9,-0.422126,-0.009999,item9,-0.271725,-0.295383,item9,-0.532875,-0.020214,item9,-0.375248,-0.102947,item9,-0.639128,-0.118726
9,item10,0.392274,0.216415,item10,0.581031,0.062171,item10,0.619554,0.681724,item10,0.755250,0.197865,item10,0.507333,0.092409,item10,0.795966,0.067398


In [36]:
files_bfi2[3]

'Human_GPT4_shape_BFI2.xlsx'

In [35]:
corrs[["feature", "mean_cor_0", "mean_cor_1", "mean_cor_2", "mean_cor_3", "mean_cor_4", "mean_cor_5"]]

,feature,feature,feature,feature,feature,feature,mean_cor_0,mean_cor_1,mean_cor_2,mean_cor_3,mean_cor_4,mean_cor_5
0,item1,item1,item1,item1,item1,item1,0.459061,0.535845,0.339805,0.550495,0.447723,0.545895
1,item2,item2,item2,item2,item2,item2,0.462582,0.563847,0.618600,0.708981,0.489179,0.670875
2,item3,item3,item3,item3,item3,item3,0.413079,0.518911,0.436846,0.513505,0.421560,0.525867
3,item4,item4,item4,item4,item4,item4,-0.151065,-0.104398,-0.239275,-0.377911,-0.167203,-0.383465
4,item5,item5,item5,item5,item5,item5,0.417086,0.428728,0.362298,0.510978,0.400531,0.706762
5,item6,item6,item6,item6,item6,item6,0.221060,0.120445,0.233079,0.334076,0.324291,0.287420
6,item7,item7,item7,item7,item7,item7,0.273263,0.499120,0.606736,0.716343,0.492310,0.718544
7,item8,item8,item8,item8,item8,item8,0.498638,0.632267,0.508197,0.671303,0.472227,0.606857
8,item9,item9,item9,item9,item9,item9,-0.255448,-0.442196,-0.333410,-0.582540,-0.362373,-0.610956
9,item10,item10,item10,item10,item10,item10,0.413252,0.584632,0.613033,0.757076,0.476802,0.798063


In [53]:
"""
可以看到，
item2, item7, item10, item13, item32, item43, item52的相关度最高，均大于0.6，并且其他模型在该item上相关度均处于-0.13到0.35之间
"""
asd = corrs[["feature", "std_cor_0", "std_cor_1", "std_cor_2", "std_cor_3", "std_cor_4", "std_cor_5"]]
asd[asd["std_cor_2"] > 0.6]

,feature,feature,feature,feature,feature,feature,std_cor_0,std_cor_1,std_cor_2,std_cor_3,std_cor_4,std_cor_5
1,item2,item2,item2,item2,item2,item2,0.146981,0.167164,0.625842,0.196623,0.094305,0.036005
6,item7,item7,item7,item7,item7,item7,-0.134368,0.105315,0.773179,0.241586,0.043737,0.111723
9,item10,item10,item10,item10,item10,item10,0.216415,0.062171,0.681724,0.197865,0.092409,0.067398
12,item13,item13,item13,item13,item13,item13,0.346919,0.279403,0.625072,0.198945,0.144989,0.062225
31,item32,item32,item32,item32,item32,item32,0.151031,0.161524,0.644853,0.241463,0.165505,0.109801
42,item43,item43,item43,item43,item43,item43,0.264579,0.273645,0.616806,0.191485,0.164825,0.081429
51,item52,item52,item52,item52,item52,item52,0.083568,0.276726,0.750586,0.237277,0.079633,0.136641


## stop test

In [57]:
df_1[["source", 'Mean Accuracy', 'Std Accuracy', 'Mean Precision', 'Std Precision',
       'Mean Recall', 'Std Recall', 'Mean F1 Score', 'Std F1 Score', 
      "Mean Coefficient (mean)", "Std Coefficient (mean)", "Mean Coefficient (std)", "Std Coefficient (std)"]]

,source,Mean Accuracy,Std Accuracy,Mean Precision,Std Precision,Mean Recall,Std Recall,Mean F1 Score,Std F1 Score,Mean Coefficient (mean),Std Coefficient (mean),Mean Coefficient (std),Std Coefficient (std)
0,Human_GPT3.5_persona_BFI2.xlsx,0.902480,0.021533,0.904509,0.021280,0.902525,0.021481,0.902353,0.021579,-0.755608,0.406495,-12.452505,0.724796
1,Human_GPT3.5_shape_BFI2.xlsx,0.740667,0.039994,0.742924,0.039190,0.740667,0.039994,0.739970,0.040487,-2.392680,0.141764,-3.772056,0.234831
2,Human_GPT4_persona_BFI2.xlsx,0.969900,0.013268,0.970457,0.012898,0.969900,0.013268,0.969886,0.013282,0.686155,0.405345,-15.041733,0.831772
3,Human_GPT4_shape_BFI2.xlsx,0.743967,0.035481,0.748554,0.034439,0.743967,0.035481,0.742667,0.036369,-1.419436,0.111477,-3.401285,0.257152
4,Human_LLaMA3_persona_BFI2.xlsx,0.653633,0.027031,0.654802,0.026904,0.653633,0.027031,0.652894,0.027402,-0.398322,0.146817,-3.023765,0.188764
5,Human_LLaMA3_shape_BFI2.xlsx,0.641026,0.034510,0.646666,0.035561,0.640533,0.034741,0.636812,0.036623,-1.771737,0.099329,-0.160031,0.087290


In [58]:
original_df

,Mean Accuracy,Std Accuracy,Mean Precision,Std Precision,Mean Recall,Std Recall,Mean F1 Score,Std F1 Score,Mean AUC,Std AUC,...,Std Squared Error (MSE),R-squared (R²) Score,Std R-squared (R²) Score,Root Mean Squared Error (RMSE),Std Root Mean Squared Error (RMSE),Mean Coefficient (mean),Std Coefficient (mean),Mean Coefficient (std),Std Coefficient (std),source
0,0.894776,0.027921,0.896756,0.027397,0.894859,0.027834,0.894648,0.027963,NaN,NaN,...,0.027921,0.579096,0.111691,0.321161,0.045602,-0.858177,0.451307,-12.075798,0.658936,Human_GPT3.5_persona_BFI2.xlsx
1,0.746667,0.036742,0.748315,0.035779,0.746667,0.036742,0.746142,0.037209,NaN,NaN,...,0.036742,-0.013333,0.146969,0.502041,0.035887,-2.390419,0.123034,-3.737859,0.200515,Human_GPT3.5_shape_BFI2.xlsx
2,0.960000,0.029533,0.962783,0.027165,0.960000,0.029533,0.959895,0.029646,NaN,NaN,...,0.029533,0.840000,0.118134,0.185075,0.075811,1.621529,0.291992,-14.791799,1.271296,Human_GPT4_persona_BFI2.xlsx
3,0.725000,0.052967,0.730160,0.055872,0.725000,0.052967,0.723679,0.052739,NaN,NaN,...,0.052967,-0.100000,0.211870,0.522180,0.048248,-1.366290,0.112722,-2.984928,0.314554,Human_GPT4_shape_BFI2.xlsx
4,0.686667,0.029155,0.688873,0.029605,0.686667,0.029155,0.685742,0.029351,NaN,NaN,...,0.029155,-0.253333,0.116619,0.559164,0.025869,-0.246231,0.097828,-3.266535,0.162881,Human_LLaMA3_persona_BFI2.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0.715000,0.034721,0.718006,0.032589,0.715000,0.034721,0.713784,0.035792,NaN,NaN,...,0.034721,-0.140000,0.138884,0.532893,0.032023,-2.203546,0.075612,-3.354736,0.191059,Human_GPT3.5_shape_BFI2.xlsx
2,0.975000,0.011785,0.975183,0.011715,0.975000,0.011785,0.974997,0.011787,NaN,NaN,...,0.011785,0.900000,0.047140,0.153040,0.039735,1.070372,0.271658,-16.709702,0.773630,Human_GPT4_persona_BFI2.xlsx
3,0.740000,0.030459,0.744058,0.030159,0.740000,0.030459,0.738820,0.031198,NaN,NaN,...,0.030459,-0.040000,0.121838,0.509068,0.029157,-1.491224,0.131259,-3.501623,0.203034,Human_GPT4_shape_BFI2.xlsx
4,0.655000,0.023333,0.656328,0.023873,0.655000,0.023333,0.654315,0.023185,NaN,NaN,...,0.023333,-0.380000,0.093333,0.587036,0.019712,-0.294072,0.119523,-2.956658,0.323026,Human_LLaMA3_persona_BFI2.xlsx


## xehao

In [59]:
df_2, original_df = get_n_rounds(files_hexao, 100, 50)

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


Optimization terminated successfully.
         Current function value: 0.352054
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.346741
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.372686
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.367424
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.383253
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.637702
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.644393
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.640877
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.639065
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.648853
  

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


Optimization terminated successfully.
         Current function value: 0.346008
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.334165
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.340100
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.336955
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.351529
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.630752
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.637429
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.638608
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.630460
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.633875
  

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


...

Optimization terminated successfully.
         Current function value: 0.314327
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.290947
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.305237
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.313776
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.303971
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.643079
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.640447
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.636835
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.635168
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.646421
  

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


Optimization terminated successfully.
         Current function value: 0.389786
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.369052
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.372164
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.369531
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.378943
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.646964
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.648155
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.643866
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.640735
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.647360
  

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_18030/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


Optimization terminated successfully.
         Current function value: 0.327170
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.301694
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.315101
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.319341
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.312724
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.624183
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.636760
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.631946
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.635072
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.641566
  

In [60]:
df_2[["source", 'Mean Accuracy', 'Std Accuracy', 'Mean Precision', 'Std Precision',
       'Mean Recall', 'Std Recall', 'Mean F1 Score', 'Std F1 Score', 
      "Mean Coefficient (mean)", "Std Coefficient (mean)", "Mean Coefficient (std)", "Std Coefficient (std)"]]

,source,Mean Accuracy,Std Accuracy,Mean Precision,Std Precision,Mean Recall,Std Recall,Mean F1 Score,Std F1 Score,Mean Coefficient (mean),Std Coefficient (mean),Mean Coefficient (std),Std Coefficient (std)
0,Human_GPT3.5_persona_HEXAO.xlsx,0.855726,0.031224,0.858068,0.031120,0.855790,0.031191,0.855492,0.031292,-1.804251,0.292099,-13.089842,0.595381
1,Human_GPT3.5_shape_HEXAO.xlsx,0.619533,0.035660,0.622855,0.036268,0.619533,0.035660,0.616919,0.036200,-1.745397,0.098809,0.347370,0.173018
2,Human_GPT4_persona_HEXAO.xlsx,0.966600,0.010155,0.967293,0.010120,0.966600,0.010155,0.966588,0.010159,-4.697200,0.914370,-20.453854,1.367742
3,Human_GPT4_shape_HEXAO.xlsx,0.677433,0.030165,0.679930,0.030881,0.677433,0.030165,0.676387,0.030172,-1.719970,0.088230,-1.392534,0.157801
4,Human_LLaMA3_persona_HEXAO.xlsx,0.556300,0.038207,0.557368,0.038937,0.556300,0.038207,0.554022,0.038954,0.583364,0.169210,3.061861,0.264370
5,Human_LLaMA3_shape_HEXAO.xlsx,0.676267,0.032238,0.683924,0.033180,0.676267,0.032238,0.672899,0.032901,-1.144710,0.030025,1.703681,0.229538


In [61]:
original_df

,Mean Accuracy,Std Accuracy,Mean Precision,Std Precision,Mean Recall,Std Recall,Mean F1 Score,Std F1 Score,Mean AUC,Std AUC,...,Std Squared Error (MSE),R-squared (R²) Score,Std R-squared (R²) Score,Root Mean Squared Error (RMSE),Std Root Mean Squared Error (RMSE),Mean Coefficient (mean),Std Coefficient (mean),Mean Coefficient (std),Std Coefficient (std),source
0,0.849510,0.021648,0.851967,0.021193,0.849520,0.021652,0.849229,0.021766,NaN,NaN,...,0.021648,0.398023,0.086588,0.386907,0.028157,-1.596233,0.161027,-12.563241,0.416653,Human_GPT3.5_persona_HEXAO.xlsx
1,0.598333,0.053333,0.600123,0.053363,0.598333,0.053333,0.595823,0.054683,NaN,NaN,...,0.053333,-0.606667,0.213333,0.632316,0.042925,-1.641683,0.108766,0.550155,0.212475,Human_GPT3.5_shape_HEXAO.xlsx
2,0.975000,0.010541,0.975861,0.010030,0.975000,0.010541,0.974986,0.010554,NaN,NaN,...,0.010541,0.900000,0.042164,0.154799,0.032205,-4.291093,0.831171,-24.374064,1.396352,Human_GPT4_persona_HEXAO.xlsx
3,0.683333,0.027386,0.686272,0.028913,0.683333,0.027386,0.682162,0.027280,NaN,NaN,...,0.027386,-0.266667,0.109545,0.562209,0.024249,-1.787702,0.050451,-1.324858,0.173302,Human_GPT4_shape_HEXAO.xlsx
4,0.558333,0.024152,0.559476,0.024041,0.558333,0.024152,0.555821,0.025399,NaN,NaN,...,0.024152,-0.766667,0.096609,0.664329,0.018278,0.341360,0.143756,2.833761,0.317999,Human_LLaMA3_persona_HEXAO.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0.641667,0.045947,0.645652,0.047451,0.641667,0.045947,0.639412,0.045942,NaN,NaN,...,0.045947,-0.433333,0.183787,0.597291,0.039713,-1.892964,0.142012,0.330942,0.150623,Human_GPT3.5_shape_HEXAO.xlsx
2,0.965000,0.008165,0.965859,0.008407,0.965000,0.008165,0.964985,0.008164,NaN,NaN,...,0.008165,0.860000,0.032660,0.185889,0.021105,-5.794740,0.643915,-19.678911,1.194386,Human_GPT4_persona_HEXAO.xlsx
3,0.680000,0.029155,0.682176,0.028434,0.680000,0.029155,0.678931,0.029699,NaN,NaN,...,0.029155,-0.280000,0.116619,0.565070,0.026391,-1.702002,0.084332,-1.364117,0.119265,Human_GPT4_shape_HEXAO.xlsx
4,0.563333,0.043653,0.564820,0.044645,0.563333,0.043653,0.560678,0.044192,NaN,NaN,...,0.043653,-0.746667,0.174611,0.659963,0.033406,0.443662,0.193841,2.874995,0.326025,Human_LLaMA3_persona_HEXAO.xlsx


## why persona gpt 4 is better for xehao

In [45]:
file = files_hexao[2]
items_count = 100
df = pd.read_excel(f'data/{file}')
df = clean(df, items_count)

df_lm = df[df["label"] == "LM"].reset_index(drop=True)
print(len(df_lm))
df_human = df[df["label"] == "Human"].reset_index(drop=True)
df_human = df_human.sample(300).reset_index(drop=True)  # we only have 300 for LM

df = pd.concat([df_lm, df_human], axis=0)


X = df[['mean', 'std']]
y = df["label"]


# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# print(y_encoded) 

# Initialize the logistic regression model (multinomial for multiple categories)
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)
model.fit(X, y)

300


LogisticRegression(max_iter=200, multi_class='multinomial')

In [46]:
file

'Human_GPT4_persona_HEXAO.xlsx'

In [54]:
"""
所以mean是不太相关的
std比较相关

"""
model.coef_

array([[-1.68728366, -4.84543534]])

In [47]:
file = files_hexao[0]
items_count = 100
df = pd.read_excel(f'data/{file}')
df = clean(df, items_count)

df_lm = df[df["label"] == "LM"].reset_index(drop=True)
print(len(df_lm))
df_human = df[df["label"] == "Human"].reset_index(drop=True)
df_human = df_human.sample(300).reset_index(drop=True)  # we only have 300 for LM

df = pd.concat([df_lm, df_human], axis=0)


item, mean_cor, std_cor = [], [], []
for i in range(1, 101):
    col = f"item{i}"
    item.append(col)

    mean_cor.append(df[col].corr(df["mean"]))
    std_cor.append(df[col].corr(df["std"]))

corrs = pd.DataFrame({"feature":item, "mean_cor_0":mean_cor, "std_cor_0":std_cor})
corrs


for j in range(1, len(files_hexao)):
    file = files_hexao[j]
    df = pd.read_excel(f'data/{file}')
    df = clean(df, items_count)
    
    df_lm = df[df["label"] == "LM"].reset_index(drop=True)
    df_human = df[df["label"] == "Human"].reset_index(drop=True)
    df_human = df_human.sample(300).reset_index(drop=True)  # we only have 300 for LM
    
    df = pd.concat([df_lm, df_human], axis=0)
    
    
    item, mean_cor, std_cor = [], [], []
    for i in range(1, 101):
        col = f"item{i}"
        item.append(col)
    
        mean_cor.append(df[col].corr(df["mean"]))
        std_cor.append(df[col].corr(df["std"]))
    
    corrs_i = pd.DataFrame({"feature":item, f"mean_cor_{j}":mean_cor, f"std_cor_{j}":std_cor})
    corrs_i

    corrs = pd.concat([corrs, corrs_i], axis=1)
corrs

/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mean'] = df[columns_selected].mean(axis=1)
/var/folders/y8/nrm8nkzx76164npfy0g513nm0000gn/T/ipykernel_89576/3077378676.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std'] = df[columns_selected].std(axis=1)


298


,feature,mean_cor_0,std_cor_0,feature,mean_cor_1,std_cor_1,feature,mean_cor_2,std_cor_2,feature,mean_cor_3,std_cor_3,feature,mean_cor_4,std_cor_4,feature,mean_cor_5,std_cor_5
0,item1,0.340082,0.163939,item1,0.558209,-0.277060,item1,0.435632,0.339803,item1,0.472194,0.100135,item1,0.434455,-0.156511,item1,0.700138,-0.056226
1,item2,0.370222,0.082512,item2,0.366090,-0.107842,item2,0.372355,0.201828,item2,0.468623,0.001187,item2,0.469906,-0.224036,item2,0.484550,-0.011977
2,item3,0.389771,-0.164639,item3,0.333544,0.034388,item3,0.343382,0.032241,item3,0.503324,-0.160211,item3,0.465850,-0.142598,item3,0.529656,0.021874
3,item4,0.441997,0.195614,item4,0.253634,-0.081150,item4,0.487931,0.293820,item4,0.556082,-0.075903,item4,0.434764,-0.054320,item4,0.437239,0.053715
4,item5,-0.172919,-0.246157,item5,-0.458314,0.258144,item5,-0.278453,-0.303414,item5,-0.205376,0.006528,item5,0.037072,-0.079091,item5,-0.220006,-0.024836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,item96,0.350814,0.262399,item96,0.576754,-0.168405,item96,0.459335,0.347510,item96,0.441480,-0.131579,item96,0.325643,-0.197094,item96,0.630078,-0.122335
96,item97,0.182683,-0.128499,item97,0.605841,-0.052073,item97,0.601454,0.584745,item97,0.772859,-0.021363,item97,0.546750,-0.059035,item97,0.833266,-0.049585
97,item98,0.381363,0.132036,item98,0.597541,-0.029935,item98,0.495059,0.401755,item98,0.738121,-0.089882,item98,0.593319,-0.214488,item98,0.738054,-0.039611
98,item99,0.195186,-0.181002,item99,0.407766,0.011689,item99,0.348562,0.166858,item99,0.640300,0.007496,item99,0.371134,-0.027574,item99,0.634755,-0.028308


In [49]:
"""
可以看到，
item8, item28, item43, item97的相关度最高，均大于0.5，并且其他模型在该item上相关度均处于-0.15到0.3之间
"""
asd = corrs[["feature", "std_cor_0", "std_cor_1", "std_cor_2", "std_cor_3", "std_cor_4", "std_cor_5"]]
asd[asd["std_cor_2"] > 0.5]

,feature,feature,feature,feature,feature,feature,std_cor_0,std_cor_1,std_cor_2,std_cor_3,std_cor_4,std_cor_5
7,item8,item8,item8,item8,item8,item8,0.272686,-0.129794,0.529472,0.093539,0.012591,0.038742
27,item28,item28,item28,item28,item28,item28,0.527768,-0.228420,0.717897,-0.111802,0.048643,-0.070638
42,item43,item43,item43,item43,item43,item43,0.295955,-0.108985,0.578736,0.026167,0.190555,-0.032940
96,item97,item97,item97,item97,item97,item97,-0.128499,-0.052073,0.584745,-0.021363,-0.059035,-0.049585


# Combined

In [62]:
def read_clean_llm(files, item_counts):
    columns_selected = [f"item{i}" for i in range(1, item_counts+1)]  

    # get LM data from each excel and combine with human data
    df = pd.read_excel(f'data/{files[0]}')
    df = df[df["label"] == "LM"].reset_index(drop=True)
    X = df[columns_selected]  # Features (34 columns)
    mask = X.apply(lambda x: x.between(1, 5)).all(axis=1)
    df = df[mask]
    df["source"] = files[0]

    for i in range(1, len(files)):
        dfi = pd.read_excel(f'data/{files[i]}')
        dfi = dfi[dfi["label"] == "LM"].reset_index(drop=True)
        X = dfi[columns_selected]  # Features (34 columns)
        mask = X.apply(lambda x: x.between(1, 5)).all(axis=1)
        dfi = dfi[mask]
        dfi["source"] = files[i]

        df = pd.concat([df, dfi], axis=0)
    print(df['label'].value_counts())
        

    # Calculate mean and standard deviation
    df['mean'] = df[columns_selected].mean(axis=1)
    df['std'] = df[columns_selected].std(axis=1)
    return df

def combine_all_lm_human(files, items_count):
    # Human data are the same acorss different files, so we fetch human from one file and LLM from others
    columns_selected = [f"item{i}" for i in range(1, items_count+1)]
    
    df_lm = read_clean_llm(files, items_count)
    
    df_human = pd.read_excel(f'data/{files[0]}')
    df_human = df_human[df_human["label"] == "Human"].reset_index(drop=True)
    df_human["source"] = "Human"
    df_human['mean'] = df_human[columns_selected].mean(axis=1)
    df_human['std'] = df_human[columns_selected].std(axis=1)

    df = pd.concat([df_lm, df_human], axis=0)
    print(df["label"].value_counts())

    return df


def pick_sample(df, human_counts, lm_counts):
    df_human = df[df["label"] == "Human"].reset_index(drop=True)
    df_human = df_human.sample(human_counts).reset_index(drop=True)

    df_lm = df[df["label"] == "LM"].reset_index(drop=True)
    df_lm = df_lm.sample(lm_counts).reset_index(drop=True)
    return pd.concat([df_lm, df_human], axis=0)

In [63]:
def get_n_rounds_combined(df, human_counts, lm_counts, items_count, n=1):
    all_results = []

    for _ in range(n):
        df = pick_sample(df, human_counts=human_counts, lm_counts=lm_counts)
        
        res = random_regression_with_kfold(df, k=5, drawing=False)
        res["source"] = "Combined"

        all_results.append(pd.DataFrame(res))
    
    # Calculate average of each metric over all rounds
    # Concatenate all results and group by 'source' to calculate mean and std of numeric columns
    all_results_df = pd.concat(all_results)
    numeric_cols = all_results_df.select_dtypes(include=np.number).columns
    averaged_results = all_results_df.groupby('source')[numeric_cols].mean().reset_index()

    return averaged_results

## BFI

In [70]:
df = combine_all_lm_human(files_bfi2, 60)
df["label"].value_counts()

label
LM    1796
Name: count, dtype: int64
label
LM       1796
Human    1559
Name: count, dtype: int64


label
LM       1796
Human    1559
Name: count, dtype: int64

In [71]:
df_1_c = get_n_rounds_combined(df, human_counts=300, lm_counts=300, items_count=60, n=50)

Optimization terminated successfully.
         Current function value: 0.539819
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.498772
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.527600
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.529338
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.529120
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.534079
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.530254
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.523518
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.533895
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.505153
  

In [72]:
df_1_c[["source", 'Mean Accuracy', 'Std Accuracy', 'Mean Precision', 'Std Precision',
       'Mean Recall', 'Std Recall', 'Mean F1 Score', 'Std F1 Score', 
      "Mean Coefficient (mean)", "Std Coefficient (mean)", "Mean Coefficient (std)", "Std Coefficient (std)"]]

,source,Mean Accuracy,Std Accuracy,Mean Precision,Std Precision,Mean Recall,Std Recall,Mean F1 Score,Std F1 Score,Mean Coefficient (mean),Std Coefficient (mean),Mean Coefficient (std),Std Coefficient (std)
0,Combined,0.742433,0.029554,0.744877,0.029773,0.742433,0.029554,0.741787,0.0297,-1.267041,0.131401,-3.706832,0.168336


## XEHAO

In [73]:
df = combine_all_lm_human(files_hexao, 100)
df["label"].value_counts()

label
LM    1798
Name: count, dtype: int64
label
Human    7204
LM       1798
Name: count, dtype: int64


label
Human    7204
LM       1798
Name: count, dtype: int64

In [74]:
df_2_c = get_n_rounds_combined(df, human_counts=300, lm_counts=300, items_count=100, n=50)

Optimization terminated successfully.
         Current function value: 0.628667
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.626834
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.622445
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.622051
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.622048
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.620005
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633318
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.629319
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.617173
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621385
  

In [75]:
df_2_c[["source", 'Mean Accuracy', 'Std Accuracy', 'Mean Precision', 'Std Precision',
       'Mean Recall', 'Std Recall', 'Mean F1 Score', 'Std F1 Score', 
      "Mean Coefficient (mean)", "Std Coefficient (mean)", "Mean Coefficient (std)", "Std Coefficient (std)"]]

,source,Mean Accuracy,Std Accuracy,Mean Precision,Std Precision,Mean Recall,Std Recall,Mean F1 Score,Std F1 Score,Mean Coefficient (mean),Std Coefficient (mean),Mean Coefficient (std),Std Coefficient (std)
0,Combined,0.6625,0.03522,0.66739,0.037214,0.6625,0.03522,0.6602,0.03528,-1.733709,0.120228,-1.536547,0.126031


In [3]:
2**85 / 10**25

3.8685626227668135

In [1]:
a = ""
a.split(".")

['']